<a href="https://colab.research.google.com/github/TRegan5/Home_sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [808 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,691 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,082 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,974 kB]
Hit:13 https:/

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

In [4]:
home_sales_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [5]:
# Check dataframe data types
home_sales_df.dtypes

[('id', 'string'),
 ('date', 'string'),
 ('date_built', 'string'),
 ('price', 'string'),
 ('bedrooms', 'string'),
 ('bathrooms', 'string'),
 ('sqft_living', 'string'),
 ('sqft_lot', 'string'),
 ('floors', 'string'),
 ('waterfront', 'string'),
 ('view', 'string')]

In [6]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')


In [26]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql('''
  SELECT YEAR(date) AS year_sold,
  ROUND(AVG (price), 2) AS average_price
  FROM home_sales
  WHERE bedrooms = 4
  GROUP BY year_sold
  ORDER BY year_sold ASC
  ''').show()

+---------+-------------+
|year_sold|average_price|
+---------+-------------+
|     2019|     300263.7|
|     2020|    298353.78|
|     2021|    301819.44|
|     2022|    296363.88|
+---------+-------------+



In [25]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql('''
  SELECT YEAR(date_built) AS year_built,
  ROUND(AVG (price), 2) AS average_price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3
  GROUP BY year_built
  ORDER BY year_built ASC
  ''').show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [24]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql('''
  SELECT YEAR(date_built) as year_built,
  ROUND(AVG (price), 2) AS average_price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 and sqft_living >= 2000
  GROUP BY year_built
  ORDER BY year_built ASC
  ''').show()



+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [23]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

query = '''
  SELECT ROUND(AVG (price), 2) AS average_price, INT(view) as view_rating
  FROM home_sales
  WHERE price >= 350000
  GROUP BY view_rating
  ORDER BY view_rating DESC
  '''

start_time = time.time()
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+-----------+
|average_price|view_rating|
+-------------+-----------+
|    1026669.5|        100|
|   1061201.42|         99|
|   1053739.33|         98|
|   1129040.15|         97|
|   1017815.92|         96|
|    1054325.6|         95|
|    1033536.2|         94|
|   1026006.06|         93|
|    970402.55|         92|
|   1137372.73|         91|
|   1062654.16|         90|
|   1107839.15|         89|
|   1031719.35|         88|
|    1072285.2|         87|
|   1070444.25|         86|
|   1056336.74|         85|
|   1117233.13|         84|
|   1033965.93|         83|
|    1063498.0|         82|
|   1053472.79|         81|
+-------------+-----------+
only showing top 20 rows

--- 0.5691983699798584 seconds ---


In [21]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [22]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data


In [ ]:
# 11. Read the parquet formatted data.


In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

